# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [69]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [70]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [71]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(full_data_rows_list[10])

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056
['Dixie Chicks', 'Logged In', 'Kaylee', 'F', '3', 'Summers', '201.01179', 'free', 'Phoenix-Mesa-Scottsdale, AZ', 'PUT', 'NextSong', '1.54034E+12', '833', "There's Your Trouble", '200', '1.54329E+12', '8']


In [72]:
# check the number of rows in your csv file

with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

    
df = pd.read_csv('event_datafile_new.csv')

df.head()
    
    

6821


,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Horn Concerto No. 4 in E flat K495: II. Romanc...,88
1,Jimi Hendrix,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Woodstock Inprovisation,88
2,Building 429,Mohammad,M,2,Rodriguez,300.61669,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Majesty (LP Version),88
3,The B-52's,Gianna,F,0,Jones,321.54077,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Love Shack,38
4,Die Mooskirchner,Gianna,F,1,Jones,169.29914,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Frisch und g'sund,38


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [73]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [74]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS tiendb 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [75]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('tiendb')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [76]:
df.head(1)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Horn Concerto No. 4 in E flat K495: II. Romanc...,88


In [77]:

query1 = "DROP TABLE song_info"

try:
    session.execute(query1)
except Exception as e:
    print(e)

In [78]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query1 = "CREATE TABLE IF NOT EXISTS song_info"
query1 = query1 + " (  song text , length float , itemInSession int , sessionId int, artist text, PRIMARY KEY (sessionId,  itemInSession, artist))"

try:
    session.execute(query1)
except Exception as e:
    print(e)


In [79]:
df = pd.read_csv('event_datafile_new.csv')
df.info()

df.userId.is_unique

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6820 entries, 0 to 6819
Data columns (total 11 columns):
artist           6820 non-null object
firstName        6820 non-null object
gender           6820 non-null object
itemInSession    6820 non-null int64
lastName         6820 non-null object
length           6820 non-null float64
level            6820 non-null object
location         6820 non-null object
sessionId        6820 non-null int64
song             6820 non-null object
userId           6820 non-null int64
dtypes: float64(1), int64(3), object(7)
memory usage: 586.2+ KB


False

In [80]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_info (sessionId , iteminSession ,length , song , artist )"
        query = query + "VALUES (%s, %s,%s,%s, %s);"

        session.execute(query, (int(line[8]), int(line[3]) , float(line[5]), line[9], line[0]) ) 

#### Do a SELECT to verify that the data have been inserted into each table

In [81]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = "SELECT * FROM song_info WHERE sessionId = 338 AND itemInSession = 4; "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.length, row.song, row.artist)

495.30731201171875 Music Matters (Mark Knight Dub) Faithless


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [82]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182


query1 = "DROP TABLE user_info; "
try:
    session.execute(query1)
except Exception as e:
    print(e)

query1 = "CREATE TABLE IF NOT EXISTS user_info (  artist text , song text , itemInSession int , userid int , firstname text , \
lastname text, sessionId int , PRIMARY KEY ((userid), sessionid, itemInSession , firstname, lastname)) WITH CLUSTERING ORDER BY ( sessionid ASC, itemInSession DESC) ;"
try:
    session.execute(query1)
except Exception as e:
    print(e)
                    
        
file = 'event_datafile_new.csv'

with open(file, encoding='utf8') as df:
    csvreader = csv.reader(df)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO user_info (artist, song, itemInSession, userid, firstname, lastname, sessionId)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"

        session.execute(query, (line[0], line[9], int(line[3]), int(line[10]), line[1], line[4], int(line[8])))


query = "SELECT * FROM user_info WHERE userid = 10 AND sessionid = 182   ;"        
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
sep = ','
for row in rows:
    print(sep.join((row.artist, row.song ,row.firstname, row.lastname)))

Error from server: code=2200 [Invalid query] message="unconfigured table user_info"
Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio Edit),Sylvie,Cruz
Sebastien Tellier,Kilometer,Sylvie,Cruz
Three Drives,Greece 2000,Sylvie,Cruz
Down To The Bone,Keep On Keepin' On,Sylvie,Cruz


In [84]:
## Getting every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query1 = "DROP TABLE song_historical; "
try:
    session.execute(query1)
except Exception as e:
    print(e)

query1 = "CREATE TABLE IF NOT EXISTS song_historical (  artist text , song text , itemInSession int , userid int , firstname text , \
lastname text, sessionId int , PRIMARY KEY ((song), sessionid, itemInSession , firstname, lastname)) WITH CLUSTERING ORDER BY ( sessionid ASC, itemInSession DESC) ;"
try:
    session.execute(query1)
except Exception as e:
    print(e)
                    
        
file = 'event_datafile_new.csv'

with open(file, encoding='utf8') as df:
    csvreader = csv.reader(df)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO song_historical (artist, song, itemInSession, userid, firstname, lastname, sessionId)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"

        session.execute(query, (line[0], line[9], int(line[3]), int(line[10]), line[1], line[4], int(line[8])))

query = "SELECT * FROM song_historical WHERE song = 'All Hands Against His Own' ;"        
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
sep = ','
for row in rows:
    print(sep.join((row.artist, row.song ,row.firstname, row.lastname)))
                    

Error from server: code=2200 [Invalid query] message="unconfigured table song_historical"
The Black Keys,All Hands Against His Own,Sara,Johnson
The Black Keys,All Hands Against His Own,Jacqueline,Lynch
The Black Keys,All Hands Against His Own,Tegan,Levine


### Drop the tables before closing out the sessions

In [85]:

# DROPPING all tables
session.execute("DROP TABLE song_historical; ")
session.execute("DROP TABLE user_info; ")
session.execute("DROP TABLE song_info; ")



### Close the session and cluster connection¶

In [86]:
session.shutdown()
cluster.shutdown()